In [20]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [21]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
print(device)

cpu


In [23]:
transformer=transforms.Compose([
    #auf 150x150 Pixel skalieren
    transforms.Resize((150,150)),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # pixelrange (of each colour channel) from 0-255 to 0-; datatyp of image is now tensor,not numpy
    transforms.Normalize([0.5,0.5,0.5], #range from 0-1 to [-1,1]
                        [0.5,0.5,0.5])
]
)

In [24]:
train_path='/home/b1/CNN/DataGeneration_v1_CNN/dataset_1/train'
test_path='/home/b1/CNN/DataGeneration_v1_CNN/dataset_1/test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    # batch size should fit to the CPU-power
    batch_size=256, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    # batch size should fit to the CPU-power
    batch_size=256, shuffle=True
)

In [25]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [26]:
print(classes)

['car', 'noCar']


In [27]:
 class ConvNet(nn.Module):
        def __init__(self,num_classes=2):
            super(ConvNet,self).__init__()
            
            # Input shape = (256,3,150,150); batch_size,numberOfChannels(rgb),height,width
            # in_channels= rgb
           
            self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
            # Output size after conv. filter ((width-kernel_size+2*Padding)/stride) +1
            # new shape: (256,12,150,150)
            self.bn1=nn.BatchNorm2d(num_features=12)
            # new shape: (256,12,150,150)
            self.relu1=nn.ReLU()
            # new shape: (256,12,150,150)
            self.pool=nn.MaxPool2d(kernel_size=2) # reduce the image size with factor 2
            # new shape: (256,12,75,75)
            
            
            self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
            # new shape: (256,20,75,75)
            self.relu2=nn.ReLU()
            # new shape: (256,20,75,75)
            
            self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
            # new shape: (256,32,75,75)
            self.bn3=nn.BatchNorm2d(num_features=32)
            # new shape: (256,32,75,75)
            self.relu3=nn.ReLU()
            # new shape: (256,32,75,75)
            
            # last layer:
            self.fc=nn.Linear(in_features=32*75*75, out_features=num_classes)
            
            #Feed forward function
        def forward(self,input):
            output=self.conv1(input)
            output=self.bn1(output)
            output=self.relu1(output)
                
            output=self.pool(output)
                
            output=self.conv2(output)
            output=self.relu2(output)
                
            output=self.conv3(output)
            output=self.bn3(output)
            output=self.relu3(output)
                
            # output will be in matrix form, with shape (256,32,75,75)
                
            output=output.view(-1,32*75*75) #last shape
                
            output=self.fc(output)
                
            return output
                
                
            
            


In [28]:
model=ConvNet(num_classes=2).to(device)

In [29]:
#optimizer+loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [30]:
num_epochs=10
#size of training/testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

1114 201


In [34]:
# save model with best epoch/accuracy

best_accuracy=0.0

for epoch in range(num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    # Model evaluation
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    #save model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(0.0392) Train Accuracy: 0.9901256732495511 Test Accuracy: 0.8706467661691543
Epoch: 1 Train Loss: tensor(0.0130) Train Accuracy: 0.9955116696588869 Test Accuracy: 0.8706467661691543
Epoch: 2 Train Loss: tensor(0.0081) Train Accuracy: 0.9964093357271095 Test Accuracy: 0.8905472636815921
Epoch: 3 Train Loss: tensor(0.0059) Train Accuracy: 0.9973070017953322 Test Accuracy: 0.900497512437811
Epoch: 4 Train Loss: tensor(0.0004) Train Accuracy: 1.0 Test Accuracy: 0.900497512437811
Epoch: 5 Train Loss: tensor(0.0003) Train Accuracy: 1.0 Test Accuracy: 0.900497512437811
Epoch: 6 Train Loss: tensor(0.0015) Train Accuracy: 0.9991023339317774 Test Accuracy: 0.900497512437811
Epoch: 7 Train Loss: tensor(0.0005) Train Accuracy: 1.0 Test Accuracy: 0.900497512437811
Epoch: 8 Train Loss: tensor(6.7861e-05) Train Accuracy: 1.0 Test Accuracy: 0.900497512437811
Epoch: 9 Train Loss: tensor(6.3991e-05) Train Accuracy: 1.0 Test Accuracy: 0.900497512437811
